In [29]:
#Set up the notebook environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy.stats import pearsonr
from scipy import signal as sig
import xgboost as xgb

In [30]:
raw = scipy.io.loadmat('./datasets/leaderboard_data.mat')
ecog_1 = raw['leaderboard_ecog'][0][0]
ecog_2 = raw['leaderboard_ecog'][1][0]
ecog_3 = raw['leaderboard_ecog'][2][0]

In [31]:
def filter_data(raw_eeg, fs=1000):
    """
    Write a filter function to clean underlying data.
    Filter type and parameters are up to you. Points will be awarded for reasonable filter type, parameters and application.
    Please note there are many acceptable answers, but make sure you aren't throwing out crucial data or adversly
    distorting the underlying data!

    Input: 
        raw_eeg (samples x channels): the raw signal
        fs: the sampling rate (1000 for this dataset)
    Output: 
        clean_data (samples x channels): the filtered signal
    """
    dim = 100
    b = sig.firwin(numtaps=dim + 1, cutoff=[0.15, 200], pass_zero='bandpass', fs=fs)

    filtered_eeg = sig.filtfilt(b, 1, x=raw_eeg, axis=0)
    
    return filtered_eeg

In [32]:
def NumWins(x, fs, winLen, winDisp):
    return int(1 + (x.shape[0] - winLen * fs) / (winDisp * fs))

winLen = 100 / 1e3
winOverlap = 50 / 1e3
winDisp = winLen - winOverlap

def LineLength(x):
    return np.abs(np.diff(x, axis=0)).sum(axis=0)

def Area(x):
    return np.abs(x).sum(axis=0)

def Energy(x):
    return (x ** 2).sum(axis=0)

def ZeroCrossingMean(x):
    return ((x < x.mean(axis=0))[1:] & (x[:-1] > x.mean(axis=0)) | (x > x.mean(axis=0))[1:] & (x[:-1] < x.mean(axis=0))).sum(axis=0)

def numSpikes(x):
    #TODO: implement
    sig.find_peaks(x, height=0, distance=100)
    pass

def averageTimeDomain(x):
    #TODO: implement
    return np.mean(x, axis=0)

def averageFreqDomain(x):
    #TODO: implement
    fourier = np.fft.fft(x, axis=1)
    N = len(fourier)
    n = np.arange(N)
    T = N/1000
    freq = n/T 
    return [np.abs(fourier[(freq >= 8) & (freq <= 12)]).mean(axis=0), 
            np.abs(fourier[(freq >= 18) & (freq <= 24)]).mean(axis=0),
            np.abs(fourier[(freq >= 75) & (freq <= 115)]).mean(axis=0),
            np.abs(fourier[(freq >= 125) & (freq <= 159)]).mean(axis=0),
            np.abs(fourier[(freq >= 159) & (freq <= 175)]).mean(axis=0)]

def bandpower(x, fs, fmin, fmax):
    fs = 1000
    # win = 4 * sf
    freqs, psd = sig.welch(x, fs, axis=0, nperseg=x.shape[0])
    
    # Define delta lower and upper limits
    # fmin, fmax = 0.5, 4

    # Find intersecting values in frequency vector
    idx_delta = np.logical_and(freqs >= fmin, freqs <= fmax)
    
    from scipy.integrate import simps

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]  # = 1 / 4 = 0.25

    # Compute the absolute power by approximating the area under the curve
    delta_power = simps(psd[idx_delta], dx=freq_res, axis=0)
    
    return delta_power
    

def get_features(filtered_window, fs=1000):
    """
        Write a function that calculates features for a given filtered window. 
        Feel free to use features you have seen before in this class, features that
        have been used in the literature, or design your own!

        Input: 
        filtered_window (window_samples x channels): the window of the filtered ecog signal 
        fs: sampling rate
        Output:
        features (channels x num_features): the features calculated on each channel for the window
    """
    feat_LL = LineLength(filtered_window)
    feat_Area = Area(filtered_window)
    feat_Energy = Energy(filtered_window)
    feat_ZCM = ZeroCrossingMean(filtered_window)
    feat_TimeAvg = averageTimeDomain(filtered_window)
    feat_FreqAvg = averageFreqDomain(filtered_window)
    
    
    return np.hstack([feat_LL, feat_Area, feat_Energy, feat_ZCM, 
                      feat_Energy,
                      feat_TimeAvg, 
                      bandpower(filtered_window, 1000, 8, 12),
                      bandpower(filtered_window, 1000, 18, 24),
                      bandpower(filtered_window, 1000, 75, 115),
                      bandpower(filtered_window, 1000, 125, 159),
                      bandpower(filtered_window, 1000, 159, 175)])

In [33]:
def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):
    """
        Write a function which processes data through the steps of filtering and
        feature calculation and returns features. Points will be awarded for completing
        each step appropriately (note that if one of the functions you call within this script
        returns a bad output, you won't be double penalized). Note that you will need
        to run the filter_data and get_features functions within this function. 

        Inputs:
        raw_eeg (samples x channels): the raw signal
        fs: the sampling rate (1000 for this dataset)
        window_length: the window's length
        window_overlap: the window's overlap
        Output: 
        all_feats (num_windows x (channels x features)): the features for each channel for each time window
            note that this is a 2D array. 
    """
    raw_ecog = filter_data(raw_ecog, fs)
    
    window_disp = window_length - window_overlap
    
    all_feats = np.vstack([get_features(raw_ecog[int(i * window_disp * fs):int(i * window_disp * fs + window_length * fs), :], fs) for i in range(NumWins(raw_ecog, fs, window_length, window_overlap))])
    
    return all_feats

In [34]:
def create_R_matrix(features, N_wind):
    """ 
    Write a function to calculate the R matrix

    Input:
        features (samples (number of windows in the signal) x channels x features): 
        the features you calculated using get_windowed_feats
        N_wind: number of windows to use in the R matrix

    Output:
        R (samples x (N_wind*channels*features))
    """
    num_win = features.shape[0]
    num_channel_features = features.shape[1]
    
    # Append a copy of the first N-1 rows to the beginning of features
    features = np.vstack((features[:N_wind-1], features))
    
    R = np.zeros((num_win, N_wind * num_channel_features))
    
    for i in range(num_win):
        # Get the feature matrix for the current window
        # Resize the feature matrix and store in R
        R[i,:] = features[i:i+N_wind,:].reshape(-1)

    R = np.hstack((np.ones((R.shape[0], 1)), R))

    return R
    

In [35]:
def pack_submission(prediction_1, prediction_2, prediction_3):
    prediction_1 = np.insert(np.repeat(prediction_1, 50, axis=0), -1, [prediction_1[-1]]*50 , axis=0)
    prediction_2 = np.insert(np.repeat(prediction_2, 50, axis=0), -1, [prediction_2[-1]]*50 , axis=0)
    prediction_3 = np.insert(np.repeat(prediction_3, 50, axis=0), -1, [prediction_3[-1]]*50 , axis=0)

    prediction_1 = np.insert(prediction_1, 3, 0, axis=1)
    prediction_2 = np.insert(prediction_2, 3, 0, axis=1)
    prediction_3 = np.insert(prediction_3, 3, 0, axis=1)
    
    scipy.io.savemat('leaderboard_prediction.mat', {'predicted_dg': [[prediction_1], [prediction_2], [prediction_3]]})

In [36]:
feature_1 = get_windowed_feats(ecog_1, 1000, winLen, winOverlap)
R_1 = create_R_matrix(feature_1, 5)
feature_2 = get_windowed_feats(ecog_2, 1000, winLen, winOverlap)
R_2 = create_R_matrix(feature_2, 5)
feature_3 = get_windowed_feats(ecog_3, 1000, winLen, winOverlap)
R_3 = create_R_matrix(feature_3, 5)

In [37]:
rf_reg_1 = pickle.load(open('./models/RF_Matrix_S1.pth', 'rb'))
prediction_1 = rf_reg_1.predict(R_1)

ValueError: X has 3411 features, but RandomForestRegressor is expecting 1861 features as input.

In [ ]:
rf_reg_2 = pickle.load(open('./models/RF_Matrix_S2.pth', 'rb'))
prediction_2 = rf_reg_2.predict(R_2)

In [ ]:
rf_reg_3 = pickle.load(open('./models/RF_Matrix_S3.pth', 'rb'))
prediction_3 = rf_reg_3.predict(R_3)

In [39]:
from xgboost import XGBRegressor
xgb_reg_1 = xgb.XGBRegressor()
xgb_reg_1.load_model("./models/XGB_S1.json")

prediction_1 = xgb_reg_1.predict(R_1)

xgb_reg_2 = xgb.XGBRegressor()
xgb_reg_2.load_model("./models/XGB_S2.json")

prediction_2 = xgb_reg_2.predict(R_2)

xgb_reg_3 = xgb.XGBRegressor()
xgb_reg_3.load_model("./models/XGB_S3.json")

prediction_3 = xgb_reg_3.predict(R_3)

In [40]:
pack_submission(prediction_1, prediction_2, prediction_3)